In [75]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

In [76]:
dataset = pd.read_csv("dublinbikes_20200101_20200401.csv")

In [77]:
df = dataset
eval_df = df.copy()
eval_df['TIME'] = pd.to_datetime(eval_df['TIME'])
eval_df['date'] = eval_df['TIME'].dt.floor('D')

eval_df = eval_df.groupby(['STATION ID','date', 'STATUS', 'ADDRESS']).agg({'AVAILABLE BIKES': 'sum', 'AVAILABLE BIKE STANDS': 'sum'})
eval_df = eval_df.reset_index(level=[0,1,2,3])
#address = eval_df['ADDRESS'].unique()
#eval_df.set_index('date', inplace=True)
#eval_df.groupby('STATION ID')['AVAILABLE BIKES'].plot(legend=True)
top_address = eval_df.groupby('ADDRESS')['AVAILABLE BIKE STANDS'].sum().nlargest(20).index.tolist()
top_df = pd.DataFrame(columns=eval_df.columns)
for address in top_address:
    res = eval_df[eval_df['ADDRESS'] == address]
    top_df = top_df.append(res)

# normal plot show
top_df.set_index('date', inplace=True)
#top_df.groupby('ADDRESS')['AVAILABLE BIKES'].plot(Legend=True)

#fig = plt.figure(figsize=(10, 6))
#ax = fig.add_subplot(2,2)
fig, ax = plt.subplots(2, 2)
#ax = plt.figure(1).add_subplot(111)
# minimize the gap between time interval
def plot_graph(ax, top_address, label):
    for address in top_address:
         ax.plot(top_df[top_df['ADDRESS'] == address].index, top_df[top_df['ADDRESS'] == address]['AVAILABLE BIKE STANDS'], label = address)
    ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    fig.autofmt_xdate()
    ax.grid(True)
    ax.set_title(f'top {label} dublin station against cycle count')
    ax.set_xlabel("Day")
    ax.set_ylabel("Total Cycle Usage")
    ax.legend(loc="upper left")
    return ax
ax[0,0] = plot_graph(ax[0,0], top_address[0:5],'5')
ax[0,1] = plot_graph(ax[0,1], top_address[5:10], '5-10')
ax[1,0] = plot_graph(ax[1,0], top_address[10:15], '10-15')
ax[1,1] = plot_graph(ax[1,1], top_address[15:20], '15-20')
fig.tight_layout()
#ax[0,0].xaxis.set_tick_params(labelbottom=True)
#ax[0,1].xaxis.set_tick_params(labelbottom=True)
#ax[1,0].xaxis.set_tick_params(labelbottom=True)
#ax[1,1].xaxis.set_tick_params(labelbottom=True)
#plt.setp(ax.get_xticklabels(), visible=True)
#for o in ax.flatten():
#    o.xaxis.set_tick_params(labelbottom=True)
#    o.yaxis.set_tick_params(labelleft=True)
fig.subplots_adjust(hspace=0.2, wspace = 0.2)

#for o in ax.flatten():
#    o.xaxis.set_tick_params(labelbottom=True)
#    o.yaxis.set_tick_params(labelleft=True)

#plt.subplot_tool()
plt.show()

In [78]:
%matplotlib qt 
df1 = df.copy()
df1['STATUS'] = df1['STATUS'].astype('category')
df1['LAST UPDATED'] = pd.to_datetime(df1['LAST UPDATED'])
df1['TIME'] = pd.to_datetime(df1['TIME'])
df1['formatted_time'] = df1['TIME'].dt.floor('h')
#ts = pd.to_datetime('2020/01/01')
#te = pd.to_datetime('2020/01/07')
#mask = (df1['TIME'] >= ts) & (df1['TIME'] <= te)
#df1.loc[mask]
#df['TIME'].max() - df['TIME'].min()

df1['day_of_week'] = df1['formatted_time'].dt.strftime('%A')
df1['day_of_month'] = df1['formatted_time'].dt.strftime('%d').astype(np.int64)
df1['hour'] = df1['formatted_time'].dt.strftime('%H').astype(np.int64)
df1['month'] = df1['formatted_time'].dt.strftime('%m').astype(np.int64)
df1['week'] = df1['formatted_time'].dt.strftime('%w').astype(np.int64)
numeric_columns = df1.select_dtypes(['int64']).columns
print(df1[numeric_columns].describe().T)
print(df1.select_dtypes(['category']).describe().T)
print(df1.dtypes)

                           count       mean        std   min   25%   50%  \
STATION ID             2228278.0  60.196628  33.598271   2.0  31.0  61.0   
BIKE STANDS            2228278.0  32.184342   7.666891  16.0  29.0  30.0   
AVAILABLE BIKE STANDS  2228278.0  20.456110  11.140868   0.0  12.0  20.0   
AVAILABLE BIKES        2228278.0  11.644086   9.980968   0.0   3.0  10.0   
day_of_month           2228278.0  15.902037   9.262035   1.0   8.0  16.0   
hour                   2228278.0  11.556068   6.876381   0.0   6.0  12.0   
month                  2228278.0   2.317890   0.729879   1.0   2.0   2.0   
week                   2228278.0   2.991054   1.976522   0.0   1.0   3.0   

                        75%    max  
STATION ID             90.0  117.0  
BIKE STANDS            40.0   40.0  
AVAILABLE BIKE STANDS  29.0   41.0  
AVAILABLE BIKES        18.0   40.0  
day_of_month           24.0   31.0  
hour                   17.0   23.0  
month                   3.0    4.0  
week               

In [66]:
ts = pd.to_datetime('2020/01/15')
te = pd.to_datetime('2020/03/10')
mask = (df1['TIME'] >= ts) & (df1['TIME'] <= te)
pd.options.mode.chained_assignment = None
#df['TIME'].max() - df['TIME'].min()
# suburb_point = "Merrion Square South"
suburb_point = "Grangegorman Lower (South)"
suburb_df = df1.loc[mask]
suburb_df = suburb_df[suburb_df['ADDRESS'] == suburb_point]
suburb_dataset = suburb_df[['TIME', 'AVAILABLE BIKES']]
#fig, ax = plt.subplots()

suburb_dataset['date'] = suburb_dataset['TIME'].dt.floor('T')
suburb_dataset = suburb_dataset.reset_index()
suburb_dataset.drop('TIME',axis=1, inplace=True)
time = suburb_dataset['date']
bikes = suburb_dataset['AVAILABLE BIKES']
#plot 2 week data for Merrion Square South region
fig = plt.figure()
ax = fig.add_subplot(111)
ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.scatter(time,bikes, color="red", marker=".", label='bike count')
ax.set_xlabel("date")
ax.set_ylabel("bike available")
fig.autofmt_xdate()
ax.grid(True)
ax.set_title(f'Bike usage for {suburb_point}')
plt.show()
suburb_df

,STATION ID,TIME,LAST UPDATED,NAME,BIKE STANDS,AVAILABLE BIKE STANDS,AVAILABLE BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE,formatted_time,day_of_week,day_of_month,hour,month,week
154497,103,2020-01-23 00:05:30,2020-01-23 00:04:26,GRANGEGORMAN LOWER (SOUTH),40,29,11,Open,Grangegorman Lower (South),53.354664,-6.278681,2020-01-23 00:00:00,Thursday,23,0,1,4
154498,103,2020-01-23 00:10:02,2020-01-23 00:04:26,GRANGEGORMAN LOWER (SOUTH),40,29,11,Open,Grangegorman Lower (South),53.354664,-6.278681,2020-01-23 00:00:00,Thursday,23,0,1,4
154499,103,2020-01-23 00:15:02,2020-01-23 00:14:34,GRANGEGORMAN LOWER (SOUTH),40,29,11,Open,Grangegorman Lower (South),53.354664,-6.278681,2020-01-23 00:00:00,Thursday,23,0,1,4
154500,103,2020-01-23 00:20:02,2020-01-23 00:14:34,GRANGEGORMAN LOWER (SOUTH),40,29,11,Open,Grangegorman Lower (South),53.354664,-6.278681,2020-01-23 00:00:00,Thursday,23,0,1,4
154501,103,2020-01-23 00:25:02,2020-01-23 00:24:43,GRANGEGORMAN LOWER (SOUTH),40,29,11,Open,Grangegorman Lower (South),53.354664,-6.278681,2020-01-23 00:00:00,Thursday,23,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512431,103,2020-03-09 23:35:02,2020-03-09 23:29:25,GRANGEGORMAN LOWER (SOUTH),40,39,1,Open,Grangegorman Lower (South),53.354664,-6.278681,2020-03-09 23:00:00,Monday,9,23,3,1
1512432,103,2020-03-09 23:40:02,2020-03-09 23:39:33,GRANGEGORMAN LOWER (SOUTH),40,39,1,Open,Grangegorman Lower (South),53.354664,-6.278681,2020-03-09 23:00:00,Monday,9,23,3,1
1512433,103,2020-03-09 23:45:02,2020-03-09 23:39:33,GRANGEGORMAN LOWER (SOUTH),40,39,1,Open,Grangegorman Lower (South),53.354664,-6.278681,2020-03-09 23:00:00,Monday,9,23,3,1
1512434,103,2020-03-09 23:50:02,2020-03-09 23:49:41,GRANGEGORMAN LOWER (SOUTH),40,39,1,Open,Grangegorman Lower (South),53.354664,-6.278681,2020-03-09 23:00:00,Monday,9,23,3,1


In [64]:
import math
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures



t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

q=2
lag=3
stride=1
w=math.floor(7*24*60*60/dt)
length = bikes.size - w - lag * w - q
print(bikes.size, w, lag, q, length)
XX=bikes[q:q+length:stride]
#week
for i in range(1,lag):
    X=bikes[i*w+q:i*w+q+length:stride]
    XX=np.column_stack((XX,X))
d=math.floor(24*60*60/dt)
#days
for i in range(0,lag):
    X=bikes[i*d+q:i*d+q+length:stride]
    XX=np.column_stack((XX,X))

for i in range(0,lag):
    X=bikes[i:i+length:stride]
    XX=np.column_stack((XX,X))

yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
tt=time[lag*w+w+q:lag*w+w+q+length:stride]

yy.reset_index(drop=True, inplace=True)
tt.reset_index(drop=True, inplace=True)

train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

alphas = []
coeff = []
errors = []

# Ci = 0.005
C = [0.001, 0.005, 0.01, 0.1, 1, 3, 5, 10, 50, 75, 100, 500, 1000, 2000, 10000]

for Ci in C:
    XPoly = PolynomialFeatures(4).fit_transform(XX)
    a = 1/(2*Ci)
    print(f'Alpha & Ci  Value - {a} {Ci}')
    model = Ridge(alpha= a,max_iter=7000).fit(XPoly[train], yy[train])
    coeff.append(model.coef_)
    alphas.append(a)
    y_pred = model.predict(XPoly)
    score = model.score(XPoly[test], yy[test])
    #r2_score = r2_score(yy,y_pred)
    from sklearn.model_selection import cross_val_score
    # scores = cross_val_score(model, XX[test], yy[test], cv=5, scoring='f1')
    mse = mean_squared_error(yy,y_pred)
    errors.append(np.sqrt(mse))
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
   .format(score,  mse,np.sqrt(mse)))
    #print(r2_score(yy,y_pred))
    print(f'r2_score - {r2_score(yy,y_pred)}')


plt.close("all")
fig, (ax1, ax2) = plt.subplots(1, 2)
#fig=plt.figure()
#ax1=fig.add_subplot(111)
ax1.plot(alphas,coeff)
ax1.set_xlabel('Alpha (Regularization Parameter)')
ax1.set_ylabel('Beta (Predictor Coefficients)')
ax1.set_title('Ridge Coefficients vs Regularization Parameters')
ax1.axis('tight')


ax2 = plt.gca()
ax2.ticklabel_format(useOffset=False)
errors = [round(num, 5) for num in errors]
ax2.plot(alphas, errors)
ax2.set_xlabel("alpha")
ax2.set_ylabel("RMSE error")
ax2.set_title("Coefficient error as a function of the regularization")
ax2.axis("tight")
fig.show()
print(alphas)
print(errors)

C:\Users\rkhna\AppData\Local\Temp/ipykernel_25840/3379008562.py:11: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


17127 2016 3 2 9061
Alpha & Ci  Value - 500.0 0.001
SCORE: 0.341, MSE:26.53, RMSE:5.15
r2_score - 0.6635775979802617
Alpha & Ci  Value - 100.0 0.005


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=9.39489e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


SCORE: -0.162, MSE:34.85, RMSE:5.90
r2_score - 0.558055504051012
Alpha & Ci  Value - 50.0 0.01


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=5.00893e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


SCORE: -0.441, MSE:39.47, RMSE:6.28
r2_score - 0.49946433338737284
Alpha & Ci  Value - 5.0 0.1


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=2.79038e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


SCORE: -0.613, MSE:42.31, RMSE:6.50
r2_score - 0.4634051100512817
Alpha & Ci  Value - 0.5 1
SCORE: -0.480, MSE:40.10, RMSE:6.33
r2_score - 0.4913600798858754
Alpha & Ci  Value - 0.16666666666666666 3
SCORE: -0.466, MSE:39.87, RMSE:6.31
r2_score - 0.49427985345820435
Alpha & Ci  Value - 0.1 5
SCORE: -0.463, MSE:39.83, RMSE:6.31
r2_score - 0.4948851027759781
Alpha & Ci  Value - 0.05 10
SCORE: -0.461, MSE:39.79, RMSE:6.31
r2_score - 0.4953438042799613
Alpha & Ci  Value - 0.01 50
SCORE: -0.460, MSE:39.76, RMSE:6.31
r2_score - 0.49571372910346245
Alpha & Ci  Value - 0.006666666666666667 75
SCORE: -0.459, MSE:39.76, RMSE:6.31
r2_score - 0.49574467564035674
Alpha & Ci  Value - 0.005 100
SCORE: -0.459, MSE:39.76, RMSE:6.31
r2_score - 0.49576015581753885
Alpha & Ci  Value - 0.001 500
SCORE: -0.459, MSE:39.75, RMSE:6.31
r2_score - 0.4957973270400958
Alpha & Ci  Value - 0.0005 1000
SCORE: -0.459, MSE:39.75, RMSE:6.31
r2_score - 0.4958019753100329
Alpha & Ci  Value - 0.00025 2000
SCORE: -0.459, MS

In [31]:
alpha = [500.0, 100.0, 50.0, 5.0, 0.5, 0.16666666666666666, 0.1, 0.05, 0.01, 0.006666666666666667, 0.005, 0.001, 0.0005, 0.00025, 5e-05]
error = [4.25234, 4.25689, 4.25785, 4.25919, 4.25959, 4.25962, 4.25963, 4.25964, 4.25964, 4.25964, 4.25964, 4.25965, 4.25965, 4.25965, 4.25965]

fig=plt.figure()
ax2=fig.add_subplot(111)
errors = [round(num, 5) for num in error]
ax2.plot(alpha, errors)
ax2.set_xlabel("alpha")
ax2.set_ylabel("RMSE error")
ax2.set_title("Coefficient error as a function of the regularization")
ax2.axis("tight")
fig.show()

In [79]:
### RIDGE MODEL TO IDENTIFY THE LAG

from sklearn.metrics import r2_score

q=2
lag_range = [1,2,3,4]
lag=3
stride=1
Ci = 0.001
scores = []
errors = []
for lag in lag_range:
    w=math.floor(7*24*60*60/dt)
    length = bikes.size - w - lag * w - q
    print("intial set : ", bikes.size, w, lag, q, length)
    XX=bikes[q:q+length:stride]
    #week
    for i in range(1,lag):
        X=bikes[i*w+q:i*w+q+length:stride]
        XX=np.column_stack((XX,X))
    d=math.floor(24*60*60/dt)
    #days
    for i in range(0,lag):
        X=bikes[i*d+q:i*d+q+length:stride]
        XX=np.column_stack((XX,X))

    for i in range(0,lag):
        X=bikes[i:i+length:stride]
        XX=np.column_stack((XX,X))

    yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
    tt=time[lag*w+w+q:lag*w+w+q+length:stride]

    yy.reset_index(drop=True, inplace=True)
    tt.reset_index(drop=True, inplace=True)

    train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

    a = 1/(2*Ci)
    print("lag value - ", lag)
    model = Ridge(alpha= a,max_iter=7000).fit(XX[train], yy[train])
    print(model.intercept_, model.coef_)
    y_pred = model.predict(XX)
    score = model.score(XX[test], yy[test])
    #r2_score = r2_score(yy,y_pred)
    from sklearn.model_selection import cross_val_score
    # scores = cross_val_score(model, XX[test], yy[test], cv=5, scoring='f1')
    mse = mean_squared_error(yy,y_pred)
    errors.append(mse)
    r2 = r2_score(yy,y_pred)
    scores.append(r2)
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
    .format(score,  mse,np.sqrt(mse)))
    #print(r2_score(yy,y_pred))
    print(f'r2_score - {r2}')

plt.close("all")
fig, (ax1, ax2) = plt.subplots(1, 2)
#fig=plt.figure()
#ax1=fig.add_subplot(111)
ax1.plot(lag_range,scores)
ax1.set_xlabel('Alpha (lag range)')
ax1.set_ylabel('Beta (Score)')
ax1.set_title('Model Score vs Lag Value')
ax1.axis('tight')

# fig=plt.figure()
# ax2=fig.add_subplot(111)

ax2 = plt.gca()
ax2.ticklabel_format(useOffset=False)
errors = [round(num, 5) for num in errors]

ax2.plot(lag_range, errors)
ax2.set_xlabel("lag")
ax2.set_ylabel("error")
ax2.set_title("Error vs Lag")
ax2.axis("tight")
fig.show()
# ax2.set_yticklabels(errors)
print(scores)
print(errors)
print(lag_range)

intial set :  12722 2016 1 2 8688
lag value -  1
4.853395747967157 [ 0.10110704  0.10110704 -0.15623877]
SCORE: -0.001, MSE:22.46, RMSE:4.74
r2_score - 0.003006759997339681
intial set :  12722 2016 2 2 6672
lag value -  2
3.755685157357906 [ 0.03631736  0.06369075  0.03631736  0.10892966  0.01848756 -0.06148289]
SCORE: 0.024, MSE:18.36, RMSE:4.29
r2_score - 0.022563839897744464
intial set :  12722 2016 3 2 4656
lag value -  3
5.401286839366507 [ 0.03769121  0.00923413 -0.0385088   0.03769121  0.11244107 -0.25500003
  0.00044622 -0.01019468  0.03769121]
SCORE: 0.077, MSE:19.26, RMSE:4.39
r2_score - 0.08421989385012707
intial set :  12722 2016 4 2 2640
lag value -  4
4.30664958031494 [ 0.0062504  -0.0478505   0.22615516 -0.17207971  0.0062504   0.0275786
 -0.1107282  -0.05639149  0.06534418  0.01150264  0.0062504  -0.10758713]
SCORE: 0.261, MSE:9.43, RMSE:3.07
r2_score - 0.2983697045641882
[0.003006759997339681, 0.022563839897744464, 0.08421989385012707, 0.2983697045641882]
[22.45916, 18

In [74]:
## IDENTIFY THE LAG ERROR

from sklearn.metrics import r2_score

q=2
lag=4
stride=1
w=math.floor(7*24*60*60/dt)
length = bikes.size - w - lag * w - q
print(bikes.size, w, lag, q, length)
XX=bikes[q:q+length:stride]
#week
for i in range(1,lag):
    X=bikes[i*w+q:i*w+q+length:stride]
    XX=np.column_stack((XX,X))
d=math.floor(24*60*60/dt)
#days
for i in range(0,lag):
    X=bikes[i*d+q:i*d+q+length:stride]
    XX=np.column_stack((XX,X))

for i in range(0,lag):
    X=bikes[i:i+length:stride]
    XX=np.column_stack((XX,X))

yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
tt=time[lag*w+w+q:lag*w+w+q+length:stride]

yy.reset_index(drop=True, inplace=True)
tt.reset_index(drop=True, inplace=True)

train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)
model = Ridge(fit_intercept=False).fit(XX[train], yy[train])
print(model.intercept_, model.coef_)
y_pred = model.predict(XX)
print("score : ", model.score(XX[test], yy[test]))

from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(yy,y_pred))
print("rmse", rmse)
print("------")
print("r2: ", r2_score(yy,y_pred))
plt.scatter(time, bikes, color="black")
plt.scatter(tt, y_pred, color="blue")
plt.xlabel("time (days)")
plt.ylabel("#bikes")
plt.legend(["training data","predictions"],loc="upper right")
day=math.floor(24*60*60/dt)
#plt.xlim((4*7,4*7+4))
plt.show()

12722 2016 4 2 2640
0.0 [-0.00059549  0.07918867  0.42216952 -0.02412458 -0.00059549  0.08871829
 -0.04102817  0.1077196   0.20709854 -0.0495786  -0.00059549 -0.09644593]
score :  0.10691950931510175
rmse 3.3392076379435083
------
r2:  0.17026693149242644
